In [ ]:
from pyspark.sql import SparkSession

**Criando uma aplicação Spark**

In [ ]:
spark = SparkSession.builder.master("local").appName("Streaming2").config("spark.jars", "/home/rowedder/Drivers/postgresql-42.3.5.jar").getOrCreate()

**Criando um Schema**

In [ ]:
jsonschema = "nome STRING, postagem STRING, data INT"

**Criando o diretório de testes**

In [ ]:
# É neste diretório que colo os arquivos json para testar. Preciso criá-lo manualmente.
df = spark.readStream.json("/home/rowedder/testestream/", schema=jsonschema)

**Criando o diretório de monitoramento da aplicação**

In [ ]:
# Não preciso criar manualmente esse diretório
diretorio = "/home/rowedder/temp/"

**Criando conexão com o PostgreSQL**

In [ ]:
def atualizapostgres(dataf, batchId):
    dataf.write.format("jdbc").option("url", "jdbc:postgresql://localhost:5432/posts").option("dbtable", "posts").option("user", "admin").option("password", "123").option("driver", "org.postgresql.Driver").mode("append").save()

In [ ]:
Stcal = df.writeStream.foreachBatch(atualizapostgres).outputMode("append").trigger(processingTime="5 second").option("checkpointlocation", diretorio).start()

In [ ]:
Stcal.awaitTermination()